In [ ]:
from vertexai.preview.language_models import ChatModel

class PaLMWrapper:
    def __init__(self):
        self.chat_model = ChatModel.from_pretrained("chat-bison@001")
        self.parameters = {
            "temperature": 0.2,
            "max_output_tokens": 256,
            "top_p": 0.95,
        }
    
    def generate_response(self, context, message):
        chat = self.chat_model.start_chat(context=context)
        response = chat.send_message(message, **self.parameters)
        return response.text

In [ ]:
pw = PaLMWrapper()
response = pw.generate_response(context='you are a london weather forecaster', message='whats the weather like today')
response

In [ ]:
import pandas as pd

import vertexai
from vertexai.preview.language_models import TextEmbeddingModel
from google.cloud import aiplatform

class VertexAIVectorStore:
    def __init__(self):
        
        # god awful way of doing it, should be a config and passed through but oh well hacky hack
        self.gen_ai_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
            index_endpoint_name="projects/playpen-8d8611/locations/europe-west2/indexEndpoints/5891412000042385408"
        )

        self.gen_ai_index = aiplatform.MatchingEngineIndex(
            index_name="projects/playpen-8d8611/locations/europe-west2/indexes/2680908415680643072"
        )
        
        self.model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
        self.df = pd.read_csv('text_data_g_embedding.csv')
        
    def search(self, input, k=3):
        embedding_vec =  self.model.get_embeddings([input])[0].values #Send request to embedding model to generate the embedding vector

        #find neighbours using vector search
        neighbours = self.gen_ai_index_endpoint.find_neighbors(
            deployed_index_id="gen_ai_deployed_index",
            queries=[embedding_vec],
            num_neighbors=k,
        )[0]
        
        results = []
        for nb in neighbours:
            text_id = nb.id
            text = self.df.iloc[int(nb.id)]['text']
            score = nb.distance
            results.append((text_id, text, score))
        
        return results


In [ ]:
vai = VertexAIVectorStore()
vector = vai.search(input='what is credit card fraud')
vector

In [ ]:
# custom RAG anyone?

class RAG:
    def __init__(self, vector_store, palm_wrapper):
        self.vector_store = vector_store
        self.palm_wrapper = palm_wrapper
    
    def generate(self, query):
        search_results = self.vector_store.search(query)
        contexts = [result[1] for result in search_results]
        
        combined_context = '\n'.join(contexts)
        response = self.palm_wrapper.generate_response(combined_context[:10000], query)
        
        return response

In [ ]:
rag = RAG(vai, pw)
query = "how do i change my address?"
response = rag.generate(query)
print(response)